# Create ML Model

## Entrada

Arquivo com todas as label preenchdidas eplas etapas anteriores

## Quando se faz Hyper-turning

É somente depois de tentar várias possibildiades com os mdelos. O hyper-turning nâo faz milagre, entao só deixa pro final.

## Descriçâo das aulas

31 - Inicio do Notebook e testando hyer-turning no RandomForest

32 - LightGBM e Sua hyperturning
+ As formas de fazer hipertuning são:
   - GridSearhc: ALém de demorada nâo tem bons resultados pois é manual e nao sabemos até que potnao estamos atingindo um bom valor o hiper-parametro que causae uma mudança positiva
   - RandomSearch: Tende a ser melhor que GridSearch
   - Baysean Optimization (AutoML): Uma nova abordagem que vamos usar nesse notebook.
     * É uma busca alertória mas guiada de forma inteligente
     
33 - Logistic Regression
+ Vamos aprender como analisar as nossa features para a regresão logistica
  - Aprensetou bons resultados
  
### Tabela de Score
````
RandomForest
 avg_precision :: 0.3876787322730141 ; roc_auc :: 0.6775564963329557
LightGBM - No Hyper-Tuning
 avg_precision :: 0.358824358569258  ; roc_auc :: 0.644678313755888
LightBGM com Baysian Optimizer
'avg_precision': 0.40242103560963294 ; 'roc': 0.6723749329199212, 
Logistic REgression = C=0.5 StandardScaler
(0.41578009994342235, 0.6897143879315486) -  
````

In [1]:
import pandas as pd
import numpy as np
import tqdm
import datetime
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

from scipy import sparse

pd.set_option("max.columns", None)

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Importar e analisar DataSet

In [3]:
df = pd.read_csv("raw_data_all_labeled2.csv", index_col=0).dropna(subset=['y'])
df.shape

(1410, 15)

In [4]:
# Sem duplicata
df.duplicated().sum()

0

In [5]:
# Sem duplicata nos titulos
df.duplicated(['title']).sum()

0

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1410 entries, 0 to 1471
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   uploader       1410 non-null   object 
 1   title          1410 non-null   object 
 2   y              1410 non-null   float64
 3   upload_date    1410 non-null   object 
 4   user           1410 non-null   object 
 5   view_count     1410 non-null   int64  
 6   like_count     0 non-null      float64
 7   dislike_count  0 non-null      float64
 8   thumbnail      1410 non-null   object 
 9   width          1410 non-null   int64  
 10  height         1410 non-null   int64  
 11  categories     1410 non-null   object 
 12  tags           1167 non-null   object 
 13  channel_url    1410 non-null   object 
 14  description    1410 non-null   object 
dtypes: float64(3), int64(3), object(9)
memory usage: 176.2+ KB


In [6]:
df.head()

,uploader,title,y,upload_date,user,view_count,like_count,dislike_count,thumbnail,width,height,categories,tags,channel_url,description
0,Yanjun Qi,S0-Introduction-Module3: Deep Learning and AI ...,0.0,2020-08-25,UCHMYETgeGbNHVHLidZSV8BQ,22,NaN,NaN,https://i.ytimg.com/vi/LkPmTGw1jqo/hqdefault.j...,1280,672,Science & Technology,Machine Learning,http://www.youtube.com/channel/UCHMYETgeGbNHVH...,Course Web: \nhttps://qiyanjun.github.io/2020f...
1,Ciência dos Dados,Machine Learning no Ensino Médio,0.0,2020-08-25,UCd3ThZLzVDDnKSZMsbK0icg,3,NaN,NaN,https://i.ytimg.com/vi_webp/R_gBq8IfwJc/maxres...,1920,1080,Education,machine learning|data science,http://www.youtube.com/channel/UCd3ThZLzVDDnKS...,"A matemática, sempre ela....\n\nDe uma maneira..."
2,iKennyHD,NBA LIVE 22: EA COULD USE DEEP MACHINE LEARNIN...,0.0,2020-08-25,KennyCallOfDuty,47,NaN,NaN,https://i.ytimg.com/vi/Tix2xon9MSs/maxresdefau...,1920,1080,Gaming,iKennyHD|nba live20|nba live 20|nba 2k20|live2...,http://www.youtube.com/channel/UCGMtoj9V9Go_im...,Wanna Donate? paypal.me/iKennyYT is where you ...
3,Amazon Web Services,Amazon Aurora Machine Learning – SageMaker Int...,0.0,2020-08-25,AmazonWebServices,335,NaN,NaN,https://i.ytimg.com/vi/w-2ip78NxAw/maxresdefau...,1920,1080,Science & Technology,AWS|Amazon Web Services|Cloud|AWS Cloud|Cloud ...,http://www.youtube.com/channel/UCd6MoB9NC6uYN2...,Learn how you can turn relational data into in...
4,"GMRIT, Rajam, AP",Machine Learning and Deep Learning Implementat...,1.0,2020-08-25,UC8g7hz4oXFzXNryt8h1gRPw,1486,NaN,NaN,https://i.ytimg.com/vi/f6XIY_M7FlA/hqdefault.j...,1280,720,People & Blogs,NaN,http://www.youtube.com/channel/UC8g7hz4oXFzXNr...,Resource Person\nMr.S.Aravinth Seshadri\nCerti...


## Data Cleaning

### 1. Limpeza das datas

In [8]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['title']
df_limpo['date'] = pd.to_datetime(df['upload_date'])

### 2. Limpeza de views

In [9]:
views = df['view_count'].fillna(0)
df_limpo['views'] = views

## Feature Engineering

In [10]:
# As features vao ficar num dataset separado
features = pd.DataFrame(index=df_limpo.index)

# Y (target) tambem fica separado
y = df['y'].copy()

# Feature Enginneering
today = datetime.datetime.today().strftime("%Y-%m-%d")
features['tempo_desde_pub'] = (pd.to_datetime(today) -  df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = (features['views'] / features['tempo_desde_pub']).round(3)
features.drop(['tempo_desde_pub'], axis=1, inplace=True)

## Split Train and Test

Vamos dividir pela data, de forma que divida ao meioa s duas quantidade de dados

In [11]:
mask_train = df_limpo['date'] < '2020-03-10'
mask_val = df_limpo['date'] >= '2020-03-10'

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((714, 2), (696, 2), (714,), (696,))

In [12]:
Xtrain.head(1)

,views,views_por_dia
253,41425,64.125


In [13]:
ytrain.head(1)

253    0.0
Name: y, dtype: float64

## Testando com apenas uma feature (NLTK)

Vamos usar um Vecotorizer para tokenizar os títulos e assim tentarmos fazer o treinamento só com palavras

In [19]:
# Buscando somente o 'title' para fazer a tokenização
title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

# Min df - minimo de vezes que palavra tem que aparecer pra virar coluna
#        => min_df=2 quer dizer enta que a palavra tem que aparecer 2 vezes
#           no minimo para ela representar uma nova coluna
## ngram_range = vai fazer colunas para tanto 1 word "machine" quanto 2 juntas "machine learning"

title_vec = TfidfVectorizer(min_df=2, ngram_range=(1,2))

## Aplicando tokenizador
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

**Convertemos o nosso titulo para um vetor de tamanho 1134, sendo esas a quantidade de palavras distintas**

Como é um array esparso (ou seja, esta preenhcido de muito zero) é difícil por isso na tela

In [23]:
title_bow_train.shape

(714, 1134)

In [22]:
title_bow_train[0][0]

<1x1134 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

## Concatenando as variávels numéricas com as geradas pelo TfidfVectorizer

In [24]:
Xtrain_wtitle = sparse.hstack([Xtrain, title_bow_train])
Xval_wtitle = sparse.hstack([Xval, title_bow_val])
Xtrain_wtitle.shape, Xval_wtitle.shape

((714, 1136), (696, 1136))

## Modelo RandomForest

In [49]:
mdl = RandomForestClassifier(n_estimators=1000,
                             min_samples_leaf=1, 
                             random_state=0, 
                             class_weight='balanced', 
                             n_jobs=4)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=4,
                       random_state=0)

In [50]:
p = mdl.predict_proba(Xval_wtitle)[: ,1]

In [51]:
print('RandomForest')
print( " avg_precision :: " + str(metrics.average_precision_score(yval, p)), '\n' ,
      "roc_auc :: " + str(metrics.roc_auc_score(yval, p)) )

RandomForest
 avg_precision :: 0.3876787322730141 
 roc_auc :: 0.6775564963329557


O melhor resultado que Mario encontrou para oe xemplo dele
min_df=2 ngram_range=(1,3)

## LightGBM

In [52]:
from lightgbm import LGBMClassifier

In [53]:
mdl = LGBMClassifier(random_state=0, class_weight='balanced', n_jobs=4)
mdl.fit(Xtrain_wtitle, ytrain)

LGBMClassifier(class_weight='balanced', n_jobs=4, random_state=0)

In [54]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

/home/rhavel/miniconda3/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


In [55]:
# metrics.average_precision_score(yval, p), metrics.roc_auc_score(yval, p)
print('LightGBM - No Hyper-turning')
print( " avg_precision :: " + str(metrics.average_precision_score(yval, p)), '\n' ,
      "roc_auc :: " + str(metrics.roc_auc_score(yval, p)) )

LightGBM - No Hyper-turning
 avg_precision :: 0.358824358569258 
 roc_auc :: 0.644678313755888


##  Bayesian Optimization

É uma opçâo avançada ao fazer GridSearch que é demorado) e RandomSearch (é melhor que o grid mas nâo tanto quanto Baysian).

**Bayesian Optimization É UM RANDOM-SEARCH OTIMIZADO**

In [34]:
# lib: scikit optimizer
from skopt import forest_minimize

In [40]:
results = []

def tune_lgbm(params):
    # Set Hyper-params
    tunning = {}
    print()
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    min_df = params[6]
    ngram_range = (1, params[7])
    # Sety TDF-Vec hyper-params, generate vocabulary and apply over data
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val)
    # Join Numeric Features with TDF-Vec
    Xtrain_wtitle = sparse.hstack([Xtrain, title_bow_train])
    Xval_wtitle = sparse.hstack([Xval, title_bow_val])
    # Create Model with hyper-params
    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth,
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1, n_estimators=n_estimators,
                        random_state=0, class_weight='balanced', n_jobs=4)
    # Fit Model
    mdl.fit(Xtrain_wtitle, ytrain)
    # Predict
    pred = mdl.predict_proba(Xval_wtitle)[:, 1]
    print(metrics.roc_auc_score(yval, pred))
    # Save Results
    tunning['params'] = params
    tunning['roc'] = metrics.roc_auc_score(yval, pred)
    tunning['avg_prec'] = metrics.average_precision_score(yval, pred)
    results.append(tunning)
    
    # Esta negativa pois eu quero maximizar a average-precision
    # como nao tem um 'skopt.forest_maximze' entao eu inverto a avg_precision para ter esse efeito de maximizar
    return -metrics.average_precision_score(yval, pred)


In [41]:
# INTERVALOS DOS HYPER-PARAMETROS  DO LIGHT-GBM E TDF-VEC
space = [(1e-3, 1e-1, 'log-uniform'), # learning-rate : usamos log-unifrom para ter mais chance de pegar valores pequenos
         (1,10), # max_depth
         (1,20), # min_child_samples
         (0.05, 1.), # subsample
         (0.05, 1.), # colsample_bytree
         (100, 1000), # n_estimators
         (1,5), # TDF-Vectorizer: min_df
         (1,5)] # TDF-Vectorizer: ngram_range

In [56]:
# EXEC skopt.forest_minimize (Baysian Optimization)
res = forest_minimize(tune_lgbm, # funçâo apra testar score que voce quer MINIMIZAR
                      space,  # hyper-paremtros e seu range
                      random_state=160745,
                      n_random_starts=20, # testar 20 vezes
                      n_calls=50, 
                      verbose=1)

In [ ]:
# ================ END OF PROCESS ==============
# Iteration No: 50 ended. Search finished for the next optimal point.
# Time taken: 22.8565
# Function value obtained: -0.3677
# Current minimum: -0.4024

In [44]:
# Melhores parametros encontrados pelo Baysian Optimizer
res.x

[0.003924937303997735,
 10,
 9,
 0.42896739513988846,
 0.08080324515701484,
 196,
 1,
 5]

In [45]:
# ALém de mostra os parametros, mostra a precision e roc auc dos melhores paremetros
sorted(results, key = lambda i: i['avg_prec'],reverse=True)[0]

{'params': [0.003924937303997735,
  10,
  9,
  0.42896739513988846,
  0.08080324515701484,
  196,
  1,
  5],
 'roc': 0.6723749329199212,
 'avg_prec': 0.40242103560963294}

## Logistic Reg

Vamos testar: Usando spo StandarScale e só MaxABsScaler. Par afazer isso, temos que comentar e descomentar os trehcos de código a seguir


In [57]:
from sklearn.preprocessing import MaxAbsScaler, StandardScaler

In [63]:
Xtrain_wtitle2 = sparse.csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = sparse.csr_matrix(Xval_wtitle.copy())

scaler = StandardScaler() ## Para variaveis numericas
# scaler = MaxAbsScaler() ## Para variaveis sparsas (o array gigante de vocabulario)

Xtrain_wtitle2[: , :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())

# Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
# Xval_wtitle2 = scaler.transform(Xval_wtitle2)

/home/rhavel/miniconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/home/rhavel/miniconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/home/rhavel/miniconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/home/r

In [64]:
Xval_wtitle2.shape

(696, 1136)

In [65]:
mdl = LogisticRegression(C=0.5, n_jobs=4, random_state=4)
mdl.fit(Xtrain_wtitle2, ytrain)

LogisticRegression(C=0.5, n_jobs=4, random_state=4)

In [66]:
p = mdl.predict_proba(Xval_wtitle2)[:, 1]

In [67]:
metrics.average_precision_score(yval, p), metrics.roc_auc_score(yval, p)

(0.41578009994342235, 0.6897143879315486)

In [ ]:
# Rafael
## (0.4147342546287094, 0.6589231411364855) - C=0.5 MaxAbScaler
## (0.41578009994342235, 0.6897143879315486) -  C=0.5 StandardScaler

In [59]:
# Mario Filho
# (0.4043414314912761, 0.6789338739490788) - sem tunning StandardScaler
# (0.3988238048468208, 0.6462226462345716) - sem tunning MaxAbScaler
# (0.33826219541849384, 0.6082881163913899) - C=10, MaxAbScaler
# (0.41472090277819385, 0.6588873650945083) - C=0.5 MaxAbScaler